In [1]:
# https://www.pyimagesearch.com/2017/03/20/imagenet-vggnet-resnet-inception-xception-keras/

import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '-1' # Set to -1 if CPU should be used CPU = -1 , GPU = 0

gpus = tf.config.experimental.list_physical_devices('GPU')
cpus = tf.config.experimental.list_physical_devices('CPU')

if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
elif cpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        logical_cpus= tf.config.experimental.list_logical_devices('CPU')
        print(len(cpus), "Physical CPU,", len(logical_cpus), "Logical CPU")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical CPU, 1 Logical CPU


In [2]:
# import the necessary packages
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import Xception # TensorFlow ONLY
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import numpy as np
import argparse
import cv2

In [5]:
# define a dictionary that maps model names to their classes
# inside Keras
MODELS = {
	"vgg16": VGG16,
	"vgg19": VGG19,
	"inception": InceptionV3,
	"xception": Xception, # TensorFlow ONLY
	"resnet": ResNet50
}
SHAPES = {
	"vgg16": (224, 224),
	"vgg19": (224, 224),
	"inception": (299, 299),
	"xception": (299, 299), # TensorFlow ONLY
	"resnet": (224, 224)
}

In [6]:
# initialize the input image shape (224x224 pixels) along with
# the pre-processing function (this might need to be changed
# based on which model we use to classify our image)
inputShape = (224, 224)
preprocess = imagenet_utils.preprocess_input
inputShape2 = (299, 299)
# if we are using the InceptionV3 or Xception networks, then we
# need to set the input shape to (299x299) [rather than (224x224)]
# and use a different image pre-processing function
#if args["model"] in ("inception", "xception"):
#	inputShape = (299, 299)
#	preprocess = preprocess_input

In [8]:
# load our the network weights from disk (NOTE: if this is the
# first time you are running this script for a given network, the
# weights will need to be downloaded first -- depending on which
# network you are using, the weights can be 90-575MB, so be
# patient; the weights will be cached and subsequent runs of this
# script will be *much* faster)

NETWORKS = {
	"vgg16": VGG16,
	"vgg19": VGG19,
	"inception": InceptionV3,
	"xception": Xception, # TensorFlow ONLY
	"resnet": ResNet50
}
for MyMODEL in MODELS:
    print("[INFO] loading {}...".format(MyMODEL))
    MyNETWORK = MODELS[MyMODEL]
    NETWORKS[MyMODEL] = MyNETWORK(weights="imagenet")

[INFO] loading vgg16...
[INFO] loading vgg19...
574717952/574710816 [==============================] - 57s 0us/step
[INFO] loading inception...
[INFO] loading xception...
91889664/91884032 [==============================] - 5s 0us/step
[INFO] loading resnet...


In [10]:
# load the input image using the Keras helper utility while ensuring
# the image is resized to `inputShape`, the required input dimensions
# for the ImageNet pre-trained network
print("[INFO] loading and pre-processing image...")
filename = 'pretrained-imagenet-models-classification/cat.jpg'
image1 = load_img(filename, target_size=(224, 224))
image1 = img_to_array(image1)
image2 = load_img(filename, target_size=(224, 224))
image2 = img_to_array(image2)

# our input image is now represented as a NumPy array of shape
# (inputShape[0], inputShape[1], 3) however we need to expand the
# dimension by making the shape (1, inputShape[0], inputShape[1], 3)
# so we can pass it through the network
image1 = np.expand_dims(image1, axis=0)
image2 = np.expand_dims(image2, axis=0)
# pre-process the image using the appropriate function based on the
# model that has been loaded (i.e., mean subtraction, scaling, etc.)
image1 = preprocess(image1)
image2 = preprocess(image2)

[INFO] loading and pre-processing image...


In [13]:
# classify the image
PREDS = {}
for MyNETWORK in NETWORKS:
    print("[INFO] classifying image with '{}'...".format(MyNETWORK))
    if SHAPES[MyNETWORK] == (224, 224):
        PREDS[MyNETWORK] = imagenet_utils.decode_predictions(NETWORKS[MyNETWORK].predict(image1))
    if SHAPES[MyNETWORK] == (299, 299):
        PREDS[MyNETWORK] = imagenet_utils.decode_predictions(NETWORKS[MyNETWORK].predict(image2))

#print("[INFO] classifying image with '{}'...".format(args["model"]))
#preds = model.predict(image)
#P = imagenet_utils.decode_predictions(preds)
# loop over the predictions and display the rank-5 predictions +
# probabilities to our terminal
for N1, P in PREDS.items():
    for (i, (imagenetID, label, prob)) in enumerate(P[0]):
        print("MODEL: {} / {}. {}: {:.2f}%".format(N1, i + 1, label, prob * 100))

[INFO] classifying image with 'vgg16'...
[INFO] classifying image with 'vgg19'...
[INFO] classifying image with 'inception'...
[INFO] classifying image with 'xception'...
[INFO] classifying image with 'resnet'...
MODEL: vgg16 / 1. Siamese_cat: 31.09%
MODEL: vgg16 / 2. wallaby: 7.81%
MODEL: vgg16 / 3. hare: 7.33%
MODEL: vgg16 / 4. wood_rabbit: 5.14%
MODEL: vgg16 / 5. doormat: 4.90%
MODEL: vgg19 / 1. Siamese_cat: 90.09%
MODEL: vgg19 / 2. Egyptian_cat: 1.91%
MODEL: vgg19 / 3. doormat: 1.56%
MODEL: vgg19 / 4. toy_terrier: 0.65%
MODEL: vgg19 / 5. weasel: 0.61%
MODEL: inception / 1. web_site: 100.00%
MODEL: inception / 2. pitcher: 0.00%
MODEL: inception / 3. saltshaker: 0.00%
MODEL: inception / 4. vase: 0.00%
MODEL: inception / 5. nematode: 0.00%
MODEL: xception / 1. ping-pong_ball: 100.00%
MODEL: xception / 2. mixing_bowl: 0.00%
MODEL: xception / 3. nipple: 0.00%
MODEL: xception / 4. necklace: 0.00%
MODEL: xception / 5. tripod: 0.00%
MODEL: resnet / 1. Egyptian_cat: 15.99%
MODEL: resnet / 2

In [16]:
# load the image via OpenCV, draw the top prediction on the image,
# and display the image to our screen

for N1, P in PREDS.items():
    for (i, (imagenetID, label, prob)) in enumerate(P[0]):
        orig = cv2.imread(filename)
        cv2.putText(orig, "MODEL: {}, Label: {}, {:.2f}%".format(N1, label, prob * 100), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
        cv2.imshow("Classification", orig)
        cv2.waitKey(0)